In [1]:
!pip install langchain
!pip install pypdf
!pip install sentence_transformers
!pip install markupsafe==2.0.1
!pip install faiss-cpu
!pip install pymupdf==1.19.0 

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple


In [2]:
# !wget -O llama_langchain.zip https://bj.bcebos.com/v1/ai-studio-online/5e882eb5d58648658950dd6ac9afb5ced4ca7a30e1f8414ba711967980b05a12?responseContentDisposition=attachment%3B%20filename%3Dllama_langchain%20.zip
# !unzip  ./llama_langchain.zip

--2023-12-29 12:10:11--  https://bj.bcebos.com/v1/ai-studio-online/5e882eb5d58648658950dd6ac9afb5ced4ca7a30e1f8414ba711967980b05a12?responseContentDisposition=attachment%3B%20filename%3Dllama_langchain%20.zip
Resolving bj.bcebos.com (bj.bcebos.com)... 111.63.96.3, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to bj.bcebos.com (bj.bcebos.com)|111.63.96.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 898022241 (856M) [application/octet-stream]
Saving to: ‘llama_langchain.zip’

llama_langchain.zip 100%[===================>] 856.42M  70.5MB/s    in 23s     

2023-12-29 12:10:34 (37.5 MB/s) - ‘llama_langchain.zip’ saved [898022241/898022241]

Archive:  ./llama_langchain.zip
replace llama_langchain/__init__.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from tqdm import tqdm 
from langchain.document_loaders import TextLoader,PyPDFLoader
from llama_langchain.models.chinese_text_splitter import ChineseTextSplitter
from typing import Any, List, Dict, Mapping, Optional
from langchain import PromptTemplate
import os

In [4]:
pdfRoot = "./pdfdata"
pdf_files_name = os.listdir(pdfRoot)

# 加载pdf
loaders = []
for pdf_file_name in pdf_files_name:
    loaders.append(PyPDFLoader(pdfRoot+"/"+pdf_file_name))

# 将pdf拆分成每一页，得到pagecontent和metedata
documents = []
for loader in loaders:
    documents += loader.load_and_split()

# 数据清洗，消除字符串里面的’\n‘
for i in range(len(documents)):
    documents[i].page_content = documents[i].page_content.replace('\n','')

# 将每一份文档拆分成若干个chunk
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 800,
    chunk_overlap  = 100,
    separators=["，","。","？","！","、"]
)

splitted_documents = text_splitter.split_documents(documents)

KeyboardInterrupt: 

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="./llama_langchain/all-mpnet-base-v2",
                                   model_kwargs={'device':"cuda"})
if not os.path.exists('./vector_store'):
    db = FAISS.from_documents(splitted_documents, embeddings)
    db.save_local('./vector_store')

In [7]:
## 本地知识库进行向量相似度匹配
CHUNK_SIZE = 800
CHUNK_OVERLAP = 100
VECTOR_SEARCH_TOP_K = 2
query = '什么是英雄史观与群众史观？'
def get_docs_with_score(docs_with_score):
    docs = []
    for doc, score in docs_with_score:
        doc.metadata["score"] = score
        docs.append(doc)
    return docs

vector_store = FAISS.load_local('./vector_store', embeddings)
vector_store.chunk_size = CHUNK_SIZE
related_docs_with_score = vector_store.similarity_search_with_score(query, k = 10)
related_docs = get_docs_with_score(related_docs_with_score)

In [8]:
related_docs

[Document(page_content='第三节人民群众在历史发展中的作用 169第三节人民群众在历史发展中的作用唯物史观有时又被称为“群众史观”。把人民群众放在至高无上的地 位，无私无我地服务和奉献于人民群众，体现了马克思主义政党和马克思 主义者的情怀和境界。那么，人民群众在社会发展中的作用如何来体现？ 历史人物在社会发展中的作用又如何来理解？一、人民群众是历史的创造者谁是历史的创造者？怎样看待人民群众和个人的历史作用？这是人类 历史上长期困扰人们的难题。唯物史观第一次科学地回答了这些问题。（一）两种历史观在历史创造者问题上的对立唯物史观与唯心史观的对立，在历史创造者问题上表现为群众史观与 英雄史观的对立。马克思主义哲学产生以前，占统治地位的历史观是英雄 史观。这种历史观从社会意识决定社会存在的基本前提出发，否认物质资 料的生产方式是社会发展的决定力量，抹杀人民群众的历史作用，宣扬少 数英雄人物创造历史。英雄史观的产生有其深刻的认识根源、社会历史根源和阶级根源。从 认识根源看，英雄史观之所以产生，主要是因为人们的认识停留在历史现 象的表面，把活跃在历史前台的少数英雄人物的作用尤其是他们的意识的 作用加以夸大并绝对化，而无视广大人民群众及其历史活动的作用。从社 会历史根源看，英雄史观的产生同社会生产力水平较低，大多数人从事物 质资料的生产活动，少数人从事政治统治、垄断精神文化生活有关。广大 人民群众在私有制社会处于被支配的地位，受剥削、受压迫，其历史创造 性得不到充分发挥和社会应有的承认，而少数剥削阶级则掌握经济、政治 权力，高高在上、独断专行，似乎可以随心所欲地主宰历史。这就为英雄', metadata={'source': './pdfdata/《马克思主义基本原理》（2023版）.pdf', 'page': 178, 'score': 0.6385535}),
 Document(page_content='，主张同外国平等交往、自由通 商，“与番人并雄”①，但严禁鸦片输入。对于外国人，强调准其勺国献策, 但不得毁谤国法。① 参见中国史学会主编：中国近代史资料丛刊《太平天国》（二），上海人民 出版社、上海书店出版社2000年版，第524、536、535、538页。', metadata={'source': './pdfdata/《中国近现代史纲要》（2

In [9]:
from langchain.llms import ChatGLM
from langchain.chains import RetrievalQA

llm = ChatGLM(
    endpoint_url='http://127.0.0.1:8000',
    max_token=80000,
    top_p=0.9
)

In [11]:
QA_CHAIN_PROMPT = PromptTemplate.from_template("""根据下面的上下文（context）内容回答问题。
如果你不知道答案，就回答不知道，不要试图编造答案。
答案最多3句话，保持答案简介。
总是在答案结束时说”谢谢你的提问！“
{context}
问题：{question}
""")
retriever = vector_store.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

ans = qa.run("什么是社会主义？")
print(ans)



> Entering new RetrievalQA chain...

> Finished chain.
社会主义的基本特征：生产资料公有制、计划经济、商品货币关系、按劳分配、生产平等、共同富裕、社会主义民主政治、社会主义核心价值体系。

谢谢你的提问！


In [47]:
professor("美国是怎么样的国家？")



> Entering new RetrievalQA chain...

> Finished chain.
本题考查美国的基本特征。

答：美国是一个资本主义国家，是世界上最大的经济体，拥有世界最发达的科技、文化和交通系统，是一个高度发达的民主国家。


In [52]:
def good_learner(query):
    ans=vector_store.similarity_search(query)
    return "emmm，我想了一会，我感觉这个知识点可能位于教材{}的第{}页,\
我背给你听嗷，好好记住吧！下次别问我了。这部分内容是：\n【{}】".format(ans[0].metadata['source'].split('/')[-1].split('.')[0],ans[0].metadata['page']+1,ans[0].page_content)

In [63]:
def teacher(query, k=10):
    embedding_vector = embeddings.embed_query(query)
    anss = vector_store.similarity_search_by_vector(embedding_vector, k=k)

    result_string = "同学啊，我刚才翻了翻书，你提到的这个问题与以下这些内容有关\n\n"
    
    for index, ans in enumerate(anss):
        result_string += "{}、{}的第{}页 ：【{}】\n\n".format(index + 1,
                                                       ans.metadata['source'].split('/')[-1].split('.')[0],
                                                       ans.metadata['page'] + 1, ans.page_content)

    result_string += "好好学习，未来是你们的！"

    return result_string

In [65]:
def professor(query):
    ans = qa.run(query)
    return ans

In [54]:
import gradio as gr

def good_learner_chatbot(message, history):
    return good_learner(message)

gr.ChatInterface(
    good_learner_chatbot,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="我是学霸，快来问我问题吧！", container=False, scale=7),
    title="思政大学霸",
    description="快来问学霸问题吧！",
    theme="soft",
    examples=["什么是社会主义？", "马克思、恩格斯", "封建社会"],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(share=True)

Caching examples at: '/root/gradio_cached_examples/195'
Caching example 1/3
Caching example 2/3
Caching example 3/3
Running on local URL:  http://127.0.0.1:7867
Running on public URL: https://f03d43ec073d848902.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [64]:
def teacher_chatbot(message, history):
    return teacher(message)

gr.ChatInterface(
    teacher_chatbot,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="我是老师，快来问我问题吧！", container=False, scale=7),
    title="思政老师",
    description="快来问老师问题吧！",
    theme="soft",
    examples=["什么是社会主义？", "马克思、恩格斯", "封建社会"],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(share=True)

Caching examples at: '/root/gradio_cached_examples/322'
Caching example 1/3
Caching example 2/3
Caching example 3/3
Running on local URL:  http://127.0.0.1:7871
Running on public URL: https://a8275c77874e474110.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new RetrievalQA chain...

> Finished chain.
社会主义是一种经济政治制度，通过生产资料公有制和按劳分配原则，实现生产关系的平等和公平，促进生产力的发展，实现共同富裕。


> Entering new RetrievalQA chain...

> Finished chain.
社会主义的基本特征：生产资料公有制、计划经济、商品经济、市场经济、资本主义市场经济、生产关系与生产力、生产目的、生产关系与生产关系、生产力与生产力、生产关系与生产目的之间的辩证关系。

谢谢你的提问！


> Entering new RetrievalQA chain...

> Finished chain.
1. 商品拜物教是什么？

2. 商品拜物教对商品生产者有什么影响？

3. 社会再生产的顺利进行需要什么条件？

4. 民主制度建设存在哪些问题？

5. 劳动成为吸引人的劳动，成为个人的自我实现，这对劳动本身有什么影响？

6. 社会主义社会，劳动会成为吸引人的劳动，成为个人的自我实现，这对劳动本身有什么影响？

7. 商品拜物教对商品生产者的影响是什么？

8. 商品拜物教对商品消费者的影响是什么？

9. 商品拜物教对商品交换的影响是什么？

10. 商品拜物教对人类社会的影响是什么？

谢谢你的提问！


In [67]:
def professor_chatbot(message, history):
    return professor(message)

gr.ChatInterface(
    professor_chatbot,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="我是教授，快来问我问题吧！", container=False, scale=7),
    title="思政教授",
    description="快来问教授问题吧！",
    theme="soft",
    examples=["什么是社会主义？", "马克思、恩格斯", "封建社会"],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(share=True)

Caching examples at: '/root/gradio_cached_examples/341'
Caching example 1/3


> Entering new RetrievalQA chain...

> Finished chain.
Caching example 2/3


> Entering new RetrievalQA chain...

> Finished chain.
Caching example 3/3


> Entering new RetrievalQA chain...

> Finished chain.
Running on local URL:  http://127.0.0.1:7872
Running on public URL: https://9d7e02b3e39bde7f02.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...


In [66]:
professor("什么是社会主义？")



> Entering new RetrievalQA chain...

> Finished chain.


'社会主义的基本特征：生产资料公有制、计划经济、商品货币关系、按劳分配、生产平等、共同富裕、生产发展、生活富裕、文明进步。\n\n谢谢你的提问！'